In [ ]:
!pip install langchain_google_genai

In [ ]:
import json
import gdown
import os

# Download the dataset through google drive share link(I made a copy from the repo)
# If the link fails, download the dataset from repo and upload to google colab workspace
folder_id = "1wyRVwbJb-8rk4pTXIs6Ny2zA1N6QM1sn?usp=drive_link"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Retrieving folder 1_eIihWAYGLL5appNIKCVp4kEnJ_2U4_C task1
Retrieving folder 1b7hdHIakf1aSOhW0pa6EYuBt6Yskn9tz 1919
Processing file 1DGAJicAWlMcs9dc29DGTf97VEsDNCRA0 S_2023_970-EN.json
Processing file 1EuZhhoI3cxrI_9fJjiWwF50tpxgGQsg5 S_2023_970-EN.pdf
Retrieving folder 1RmJ-lyEyxD57kVt_9aNanlIH8DOxPbzD 1938
Processing file 1UHGuvaECNnwD46itc3Kqhgl5QnAcJA6K S_2024_302-EN.json
Processing file 1q3LQYiflAp3EQRYn9wi7BnWqAwWJVIL- S_2024_302-EN.pdf
Retrieving folder 1ahwnsLCJpAXocsHM8bf7ZEKq4Ryd7tNb 1942
Processing file 1cTzMkh5fRdHgA26Hqo5YXAXqK3ytVE3- S_2024_400-EN.json
Processing file 1KFQ_Rge8V1BmwqyKNgfdhYkOF9cJ0MLM S_2024_400-EN.pdf
Retrieving folder 1qbYFXLTONDW1Xwlw5QZMnWM8nlp0MsKa 1956
Processing file 10aDZnHwMEZ3b8pamaCUAlpTTJnAEb2on S_2024_557-EN.json
Processing file 1pT3Z0JDGa_zFhnmPD5t1gyMjUbIJHzg1 S_2024_557-EN.pdf
Retrieving folder 1Aq1ivx8g7wkPMZgQAA1E8jnszaJmSFdh 1976
Processing file 1ACLll1QNTWQmjWwfU8ynFul3LnA-1cml S_2024_835-EN.json
Processing file 1cMAiQSquoFo1cwUx47806b3

Retrieving folder contents completed
Building directory structure
Building directory structure completed


In [ ]:
with open('../content/data/task1.json', 'r') as f:
    test = json.load(f)

In [ ]:
draft_ids = []
authors = []
coauthors = []
choices_2 = []
choices_3 = []
choices_4 = []
choices_5 = []

for instance in test:
    draft_ids.append(instance['folder_id'])
    authors.append(instance['author'])
    coauthors.append(instance['coauthor'])
    choices_2.append(instance['choices_2'])
    choices_3.append(instance['choices_3'])
    choices_4.append(instance['choices_4'])
    choices_5.append(instance['choices_5'])

In [22]:
import os

path = '../content/data/task1'
drafts = []
for i in draft_ids:
    folder_path = os.path.join(path, str(i))
    files = os.listdir(folder_path)
    json_file = [file for file in files if file.endswith('EN.json')][0]
    with open(os.path.join(folder_path, json_file)) as f:
        draft = json.load(f)
    drafts.append(draft['Content'])

In [ ]:
# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [ ]:
# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
import json, re
import random
from tqdm import tqdm

def run_task3_k_choices(drafts, authors, choices_k):
    results = []
    invalid_responses = []
    for i, (draft, author, choice_k) in tqdm(enumerate(zip(drafts, authors, choices_k))):
        SYSTEM_PROMPT = f"""You are representing {author}, a country drafting a resolution for submission to the United Nations Security Council.
        Your task is to review the draft resolution and select a coauthor from the following list: {', '.join(choice_k)}.
        Respond only with the name of the chosen coauthor and provide no additional explanation."""
        user_prompt = f"""{author} is drafting a resolution to submit to the United Nations Security Council and is seeking a coauthor.

        Review the following draft resolution and choose a coauthor from the list below.
        Respond only with the name of the chosen country and provide no additional explanation.

        Available coauthors: {', '.join(choice_k)}

        Draft Resolution:
        {draft}

        Answer:
        """

        messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=user_prompt),
        ]

        response = llm.invoke(messages)
        raw_text = response.content


        result = raw_text
        valid_response = choice_k
        if result not in valid_response:
            print(f'Invalid response: {result}')
            result = random.choice(valid_response)
            invalid_responses.append(i)
        results.append(result)
    return results, invalid_responses

In [ ]:
results_2, invalid_responses_2 = run_task3_k_choices(drafts, authors, choices_2)

30it [02:40,  5.34s/it]


In [ ]:
results_3, invalid_responses_3 = run_task3_k_choices(drafts, authors, choices_3)

30it [02:59,  5.97s/it]


In [ ]:
results_4, invalid_responses_4 = run_task3_k_choices(drafts, authors, choices_4)

13it [01:27,  6.20s/it]

Invalid response: Egypt


28it [03:19,  6.87s/it]

Invalid response: Egypt


30it [03:35,  7.18s/it]


In [ ]:
results_5, invalid_responses_5 = run_task3_k_choices(drafts, authors, choices_5)

30it [03:54,  7.81s/it]


In [ ]:
def calculate_accuracy(results, ground_truth):
    correct = 0
    for i in range(len(results)):
        if results[i] == ground_truth[i]:
            correct += 1
    return correct/len(results)

In [ ]:
accuracy_2 = calculate_accuracy(results_2, coauthors)
accuracy_3 = calculate_accuracy(results_3, coauthors)
accuracy_4 = calculate_accuracy(results_4, coauthors)
accuracy_5 = calculate_accuracy(results_5, coauthors)

In [ ]:
print(f'Accuracy for 2 choices: {accuracy_2}')
print(f'Accuracy for 3 choices: {accuracy_3}')
print(f'Accuracy for 4 choices: {accuracy_4}')
print(f'Accuracy for 5 choices: {accuracy_5}')

Accuracy for 2 choices: 0.8333333333333334
Accuracy for 3 choices: 0.6
Accuracy for 4 choices: 0.43333333333333335
Accuracy for 5 choices: 0.4666666666666667


In [ ]:
# MODIFIED: CoT Prompt version (your "small change")
def run_task1_cot_k_choices(drafts, authors, choices_k):
    results = []
    invalid_responses = []
    for i, (draft, author, choice_k) in tqdm(enumerate(zip(drafts, authors, choices_k))):
        SYSTEM_PROMPT = f"""You are {author}, drafting a UNSC resolution.
        Select ONE co-penholder from: {', '.join(choice_k)}.

        STEP 1: Identify main topics (e.g., peacekeeping, sanctions).
        STEP 2: Which candidate aligns best (region, prior alliances, expertise)?
        STEP 3: Output ONLY the chosen country name."""

        user_prompt = f"Draft: {draft}\n\nAnswer:"

        messages = [SystemMessage(content=SYSTEM_PROMPT),
               HumanMessage(content=user_prompt)]
        response = llm.invoke(messages)
        result = response.content.strip()

        if result not in choice_k:
            print(f'Invalid: {result}')
            result = random.choice(choice_k)
            invalid_responses.append(i)
        results.append(result)
    return results, invalid_responses

# Run modified (same first 30!)
cot_results_2, _ = run_task1_cot_k_choices(drafts[:30], authors[:30], choices_2[:30])
cot_results_3, _ = run_task1_cot_k_choices(drafts[:30], authors[:30], choices_3[:30])
cot_results_4, _ = run_task1_cot_k_choices(drafts[:30], authors[:30], choices_4[:30])
cot_results_5, _ = run_task1_cot_k_choices(drafts[:30], authors[:30], choices_5[:30])

# Compare
cot_acc_2 = calculate_accuracy(cot_results_2, coauthors[:30])
cot_acc_3 = calculate_accuracy(cot_results_3, coauthors[:30])
cot_acc_4 = calculate_accuracy(cot_results_4, coauthors[:30])
cot_acc_5 = calculate_accuracy(cot_results_5, coauthors[:30])



30it [03:29,  6.98s/it]
18it [02:17,  6.76s/it]

Invalid: Group of Arab States


30it [03:34,  7.14s/it]
13it [01:52,  8.40s/it]

Invalid: Egypt


14it [02:01,  8.56s/it]

...

30it [04:14,  8.49s/it]


Invalid: Venezuela (Bolivarian Republic of) Permanent Mission to the United Nations


14it [02:17,  9.39s/it]

Invalid: Netherlands


30it [04:44,  9.47s/it]


In [ ]:
print(f'CoT 1/2: {cot_acc_2}, 1/3: {cot_acc_3}, 1/4: {cot_acc_4}, 1/5: {cot_acc_5}')

CoT 1/2: 0.6666666666666666, 1/3: 0.7, 1/4: 0.36666666666666664, 1/5: 0.26666666666666666
